In [ ]:
# Install necessary libraries
!pip install fuzzywuzzy[speedup] openpyxl pandas

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz, process
from google.colab import files

In [ ]:
# Step 1: Upload the Excel files from your local system
print("Please upload 'Mark Meyers.xlsx'")
uploaded = files.upload()

print("Please upload 'Shauns Report.xlsx'")
uploaded = files.upload()

Please upload 'Mark Meyers.xlsx'


Saving Mark Meyers.xlsx to Mark Meyers.xlsx
Please upload 'Shauns Report.xlsx'


Saving Shauns Report.xlsx to Shauns Report.xlsx


In [ ]:
# Load the Excel files from the uploaded data
mark_file = 'Mark Meyers.xlsx'  # This is the uploaded file name
shaun_file = 'Shauns Report.xlsx'  # This is the uploaded file name

In [ ]:
# Use pd.ExcelFile to load the workbook
excel_data = pd.ExcelFile(mark_file)

# List all the sheet names in the workbook
sheet_names = excel_data.sheet_names
print("Available sheets in the file:")
print(sheet_names)

Available sheets in the file:
['By_Bill_To_Company', 'By_Ship_To_Company', 'By Category', 'By Profit', 'Group Summary', 'Report Specifications']


In [ ]:

excel_file = pd.ExcelFile(mark_file)

# Get sheet names
sheet_names = excel_file.sheet_names

# Get column names for each sheet
sheet_columns = {}
for sheet in sheet_names:
    df = pd.read_excel(excel_file, sheet_name=sheet)
    sheet_columns[sheet] = df.columns.tolist()

# Print the column names for each sheet
for sheet, columns in sheet_columns.items():
    print(f"Sheet: {sheet}")
    print(f"Columns: {columns}")


Sheet: By_Bill_To_Company
Columns: ['Sales Rep', 'billcomp', 'OTS\nBillto', 'billcity', 'billstate', 'billzip', 'Product Category', 'trancount', 'Quantity', 'Total Sales', 'Total Cost', 'Total Profit', 'rptqty', 'WFS\nGroup#', 'Group Name', 'wfsaccount', 'wfscrr', 'businesstype', 'legacyname', 'fedid', 'subcategory']
Sheet: By_Ship_To_Company
Columns: ['Sales Rep', 'billcomp', 'OTS\nBillto', 'OTS\nShipto', 'shipcomp', 'shipcity', 'shipstate', 'shipzip', 'Product Category', 'trancount', 'Quantity', 'Total Sales', 'Total Cost', 'Total Profit', 'rptqty', 'WFS\nGroup#', 'Group Name', 'wfsaccount', 'wfscrr', 'businesstype', 'legacyname', 'Airport', 'fedid', 'subcategory', 'brandname', 'ICAO']
Sheet: By Category
Columns: ['Sales Rep', 'Product Category', 'subcategory', 'trancount', 'Quantity', 'Total Sales', 'Total Cost', 'Total Profit', 'rptqty']
Sheet: By Profit
Columns: ['billcomp', 'OTS\nBillto', 'Sales Rep', 'trancount', 'Quantity', 'Total Sales', 'Total Cost', 'Total Profit', 'rptqty']

In [ ]:

excel_file = pd.ExcelFile(mark_file)

# Get sheet names
sheet_names = excel_file.sheet_names

# Get unique rows for "Product Category" column for each sheet
unique_product_categories = {}
for sheet in sheet_names:
    df = pd.read_excel(excel_file, sheet_name=sheet)
    if 'Product Category' in df.columns:
        unique_product_categories[sheet] = df['Product Category'].drop_duplicates().tolist()

# Print the unique rows for "Product Category" column for each sheet
for sheet, unique_rows in unique_product_categories.items():
    print(f"Sheet: {sheet}")
    print(f"Unique Product Categories: {unique_rows}")


Sheet: By_Bill_To_Company
Unique Product Categories: ['Avgas', 'Jet Fuel', 'Software', 'Other Charges', 'Financial / BDF', 'Refuelers / Avgas', 'Refuelers / Jet', 'Signs / World', 'Other Costs', 'De-Icing Fluid', 'Misc', 'Signs / Phillips', 'Credit Cards', 'Signs / Ascent', 'Storage / Deicing St', 'Anti-Icing Additive', nan]
Sheet: By_Ship_To_Company
Unique Product Categories: ['Other Costs', 'Avgas', 'Jet Fuel', 'Software', 'Other Charges', 'Financial / BDF', 'Refuelers / Avgas', 'Refuelers / Jet', 'Signs / World', 'De-Icing Fluid', 'Misc', 'Signs / Phillips', 'Credit Cards', 'Signs / Ascent', 'Storage / Deicing St', 'Anti-Icing Additive', nan]
Sheet: By Category
Unique Product Categories: ['Anti-Icing Additive', 'Avgas', 'De-Icing Fluid', 'Jet Fuel', 'Misc', 'Software', 'Other Charges', 'Credit Cards', 'Financial / BDF', 'Other Costs', 'Refuelers / Avgas', 'Refuelers / Jet', 'Signs / Ascent', 'Signs / Phillips', 'Signs / World', 'Storage / Deicing St']
Sheet: Group Summary
Unique Pro

In [ ]:
# Load all sheets from Mark Meyers file
mark_meyers = pd.read_excel(mark_file, sheet_name=None)

# Load Shaun's Report
shauns_report = pd.read_excel(shaun_file)

In [ ]:
# Extract the relevant columns from both sheets
by_bill_to_company = mark_meyers['By_Bill_To_Company']
by_ship_to_company = mark_meyers['By_Ship_To_Company']
fbo_name_shauns_report = shauns_report['FBO Name']

In [ ]:
# Set threshold for fuzzy match (you can adjust this)
threshold = 85

# Function to perform fuzzy matching
def fuzzy_match(series_to_match, column_to_compare):
    matched_indices = []
    for idx, value in enumerate(series_to_match):
        # Extract the best match and its score
        match_data = process.extractOne(value, column_to_compare, scorer=fuzz.token_sort_ratio)
        if match_data:  # Ensure there's a match
            match, score = match_data[0], match_data[1]  # Unpack the best match and score
            if score >= threshold:
                matched_indices.append(idx)
    return matched_indices

In [ ]:
# Ensure all columns used in fuzzy matching are converted to strings and handle NaNs
fbo_name_shauns_report = fbo_name_shauns_report.fillna('').astype(str)
by_bill_to_company['billcomp'] = by_bill_to_company['billcomp'].fillna('').astype(str)
by_ship_to_company['billcomp'] = by_ship_to_company['billcomp'].fillna('').astype(str)
by_ship_to_company['shipcomp'] = by_ship_to_company['shipcomp'].fillna('').astype(str)

In [ ]:
# Perform fuzzy matching for all comparisons
bill_comp_matches = fuzzy_match(fbo_name_shauns_report, by_bill_to_company['billcomp'])
billcomp_matches = fuzzy_match(fbo_name_shauns_report, by_ship_to_company['billcomp'])
shipcomp_matches = fuzzy_match(fbo_name_shauns_report, by_ship_to_company['shipcomp'])

In [ ]:
# Combine the matched indices from all three comparisons
matched_indices = set(bill_comp_matches) | set(billcomp_matches) | set(shipcomp_matches)

# Filter Shaun's Report to keep only the matched rows
filtered_shauns_report = shauns_report.iloc[list(matched_indices)]

# Save the filtered report to a new Excel file
output_path = '/content/Filtered_Shauns_Report.xlsx'
filtered_shauns_report.to_excel(output_path, index=False)

# Provide the output path for download
print(f"Filtered report saved to: {output_path}")

Filtered report saved to: /content/Filtered_Shauns_Report.xlsx
